# Explorative Data Analysis

## Imports

In [65]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import json 
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
nltk.download('vader_lexicon')
from nltk.corpus import twitter_samples
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\lucaf\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## Data Preperation

In [66]:
#reading the json of the data
df = pd.read_json("./cryptodoge.json", lines=True)

In [67]:
#extracting followers_count from user column
df["followers_count"] = 0
for index, row in df.iterrows():
  df["followers_count"][index] = df["user"][index]["followers_count"]
#extracting location from user column
df["location"] = 0
for index, row in df.iterrows():
  df["location"][index] = df["user"][index]["location"]  
#extracting verified from user column
df["verified"] = 0
for index, row in df.iterrows():
  df["verified"][index] = df["user"][index]["verified"]  
#extracting friends_count from user column
df["friends_count"] = 0
for index, row in df.iterrows():
  df["friends_count"][index] = df["user"][index]["friends_count"]    

<ipython-input-67-bf84cfa9801d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["followers_count"][index] = df["user"][index]["followers_count"]
<ipython-input-67-bf84cfa9801d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["location"][index] = df["user"][index]["location"]
C:\Users\lucaf\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._se

In [68]:
#dropping unneeded columns
df.drop(["created_at", "id", "id_str", "source", "truncated", "in_reply_to_status_id", "in_reply_to_status_id_str", "in_reply_to_user_id_str", "in_reply_to_user_id", "in_reply_to_screen_name", "coordinates", "place", "quoted_status_id", "quoted_status_id_str", "is_quote_status", "quoted_status", "retweeted_status", "entities", "extended_entities", "favorited", "retweeted", "possibly_sensitive", "filter_level", "contributors", "geo", "user", "extended_tweet", "quoted_status_permalink", "location", "timestamp_ms", "display_text_range"], axis=1)

,text,quote_count,reply_count,retweet_count,favorite_count,lang,followers_count,verified,friends_count
0,RT @dailytice: 🎁 $100 IN BABYDOGE GIVEAWAY ALE...,0,0,0,0,en,3,False,65
1,RT @CoinMarketCap: Bitcoin 'Could Collapse' El...,0,0,0,0,en,19,False,157
2,@MattWallace888 Amen brother!\n#dogecoin \n#To...,0,0,0,0,en,74,False,159
3,@BitcoinAssn The most powerful DOGE of them al...,0,0,0,0,en,0,False,1
4,RT @RakuCoin: Rakucoin $RAKUC currently on ETH...,0,0,0,0,en,85,False,4913
...,...,...,...,...,...,...,...,...,...
1002,RT @JubaJeeves: I went back to microwaving hot...,0,0,0,0,en,147,False,194
1003,RT @RakuCoin: Rakucoin $RAKUC currently on ETH...,0,0,0,0,en,131,False,348
1004,RT @flurbnb: $120 to one person in 24 hours\n\...,0,0,0,0,en,2413,False,4943
1005,@BinanceAcademy The most powerful DOGE of them...,0,0,0,0,en,0,False,1


In [69]:
df_tweets = df

In [70]:
pos_twee = twitter_samples.strings('positive_tweets.json')
neg_twee = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')

df = pd.DataFrame (pos_twee,columns=['text'])
df['Value'] = 1
df2 = pd.DataFrame (neg_twee,columns=['text'])
df2['Value'] = -1

df=df.append(df2, ignore_index = True)
df = df.sample(frac=1).reset_index(drop=True) #shuffle neg and pos tweets 
#cleansing tweet
remove_rt = lambda x: re.sub("(RT @\w+: )|(@\w+):"," ",x)
rt = lambda x: re.sub("(@[A-Za-z0–9]+)|(\w+:\/\/\S+)|(\n|\t|\')"," ",x)


df["text"] = df.text.map(remove_rt).map(rt)
df["text"] = df.text.str.lower()
df.text.head(10)

#removing emoji characters
def deEmojify(text):
    regrex_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return regrex_pattern.sub(r'',text)

df["text"] = df.text.map(deEmojify)

data = list(df["text"])
labels = list(df["Value"])
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(data)
vectors_train = vectorizer.transform(x_train)
vectors_test = vectorizer.transform(x_test)
model = MultinomialNB(alpha=.01)
model = model.fit(vectors_train, y_train)



In [ ]:
#bereinigen der Tweets
remove_rt = lambda x: re.sub("(RT @\w+: )|(@\w+):"," ",x) #herausfiltern der usernamen und RT
rt = lambda x: re.sub("(@[A-Za-z0–9]+)|[\d\.]|(\w+:\/\/\S+)|(\n|\t|\')"," ",x) #herausfiltern markierter Usernamen 
htp = lambda x: re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'," ",x) #entfernen von links, sonderzeichen, punkten

df_tweets["text"] = df.text.map(remove_rt).map(rt).map(htp)
df_tweets["text"] = df.text.str.lower()
df_tweets.text.head(10)

#bereinigen des textes von den Emojis
def deEmojify(text):
    regrex_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return regrex_pattern.sub(r'',text)

df_tweets["text"] = df_tweets.text.map(deEmojify)